# Análise e Controle de Sistemas Lineares Incertos por meio de Desigualdades Matriciais Lineares: Exercício

## Descrição

Eeste notebook explora conceitos avançados de análise e controle de sistemas lineares incertos através da formulação e resolução de problemas de programação linear utilizando as bibliotecas CVX e Mosek em Python, com ênfase em Inequações Matriciais Lineares (LMIs).

### Data de Criação

2024-01-21

### Data de Modificação

2024-01-22

## Autor

Andevaldo da Encarnação Vitório